<a href="https://colab.research.google.com/github/aubricot/computer_vision_with_eol_images/blob/master/object_detection_for_image_cropping/aves/aves_generate_crops_yolo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Using YOLO v2 in Darkflow to detect birds from images
---
*Last Updated 16 March 2020*   
-Darkflow builds are no longer being updated. As a result, this notebook is left in its state from March 2020. Functions may become deprecated or lose functionality. For updated inference with Aves, refer to Tensorflow notebooks-

Using YOLO via Darkflow as a method to do customized, large-scale image processing. Using the location and dimensions of the detected birds, images will be cropped to square dimensions that are centered and padded around the detection box. Pre-trained models are used for "out of the box" inference on images of birds.

## Installs
---

In [ ]:
# Mount google drive to export detection results as tsv
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Change to your working directory within Google Drive
%cd drive/My Drive/train

# Make sure you are using Python 3.6
# Install packages using pip
!python --version
!pip install tensorflow-gpu==1.15.0rc2
!pip install cython
!pip install opencv-python

/content/drive/My Drive/train
Python 3.6.9
     |████████████████████████████████| 411.5MB 42kB/s 
     |████████████████████████████████| 51kB 8.8MB/s 
     |████████████████████████████████| 512kB 55.4MB/s 
     |████████████████████████████████| 3.8MB 55.0MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7540 sha256=cd8db42216007229a2c80206b695fc884fa73b38c376c7cd60e352a815b39e03
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow 2.4.1 has requirement gast==0.3.3, but you'll have gast 0.2.2 which is incompatible.
ERROR: tensorflow 2.4.1 has requirement tensorboard~=2.4, but you'll have tensorboard 1.15.0 which is incompatible.
ERROR: tensorflow 2.4.1 has requirement tensorflow-estimator<2.5.0,>=2.4.0, but you'll have tensorflow-estimator 1.15.1 which is incompatible.
ERROR: tensorflow-probability 0.12.1 has requirement gast>=0.3.2, but you'll have gast 0.2.2 whi

In [ ]:
# Download and build darkflow (the tensorflow implementation of YOLO)
import os
import pathlib

if "darkflow-master" in pathlib.Path.cwd().parts:
  while "darkflow-master" in pathlib.Path.cwd().parts:
    os.chdir('..')
elif not pathlib.Path("darkflow-master").exists():
  !git clone --depth 1 https://github.com/thtrieu/darkflow.git
  # Compile darkflow
  %cd darkflow
  !python setup.py build_ext --inplace
  # Change darkflow to darkflow-master to distinguish between folder names
  %cd ../
  !mv darkflow darkflow-master

### Imports   
---

In [ ]:
%cd darkflow-master

%tensorflow_version 1.15.0rc2
import tensorflow as tf
print(tf.__version__)

# For importing/exporting files, working with arrays, etc
import pathlib
import time
import csv
import urllib
import numpy as np
import pandas as pd

# For the actual object detection
from darkflow.net.build import TFNet

# For drawing onto and plotting the images
import matplotlib.pyplot as plt
import cv2
%config InlineBackend.figure_format = 'svg'

/content/drive/My Drive/train/darkflow-master
`%tensorflow_version` only switches the major version: 1.x or 2.x.
You set: `1.15.0rc2`. This will be interpreted as: `1.x`.


TensorFlow 1.x selected.
1.15.2
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.











### Model Preparation
---   
**Uploads**: The models are already in darkflow/cfg, but the pre-trained weights associated with these models need to be uploaded to this notebook from https://drive.google.com/drive/folders/0B1tW_VtY7onidEwyQ2FtQVplWEU. 

**"Flowing" images through the model**: Ignore the warning messages about deprecated names, they still work at the time this last updated. Code for parameters is based on https://github.com/thtrieu/darkflow ("Using darkflow from another python application").

Your output should be a table of values like those shown below:

Source | Train? | Layer description                | Output size
------- |:--------:|:----------------------------------:| ---------------
       |        | input                            | (?, 448, 448, 3)
 Load  |  Yep!  | scale to (-1, 1)                 | (?, 448, 448, 3)
 Load  |  Yep!  | conv 3x3p1_1    leaky            | (?, 448, 448, 16)

**Define boxing function**: You can adjust the parameters so that bounding boxes are only shown for certain confidence or class values. Here boxes are shown when confidence > 0.45 and object class is 'bird'. This function is modified from here https://gist.github.com/deep-diver/40f092ad56525189674a86b6fde6d304.

In [ ]:
# Test installation, you should see an output with different parameters for flow
%cd darkflow-master
!python flow --h

In [ ]:
# Upload yolo.weights, pre-trained weights file (for YOLO v2) from Google drive 
# For directions to upload other weights files, see the wiki for this repository
weights = 'yolo'
weights_file = weights + '.weights'
if not os.path.exists('weights_file'):
  !gdown --id 0B1tW_VtY7oniTnBYYWdqSHNGSUU
  !mkdir bin
  !mv yolo.weights bin

# Define parameters for "flow"ing the images through the model
# Can change detection confidence threshold here
params = {
    'model': 'cfg/yolo.cfg',
    'load': 'bin/yolo.weights',
    'threshold': 0.45, 
    'gpu': 1.0
}

# Run the model
tfnet = TFNet(params)

# For uploading an image from url
# Modified from https://www.pyimagesearch.com/2015/03/02/convert-url-to-image-with-python-and-opencv/
def url_to_image(url):
  resp = urllib.request.urlopen(url)
  image = np.asarray(bytearray(resp.read()), dtype="uint8")
  image = cv2.imdecode(image, cv2.IMREAD_COLOR)
  image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
 
  return image

# For drawing bounding boxes around detected objects on images
def boxing(image, predictions):
    newImage = np.copy(image)
    im_height, im_width, im_depth = image.shape
        
    for result in predictions:
        xmin = result['topleft']['x']
        ymin = result['topleft']['y']

        xmax = result['bottomright']['x']
        ymax = result['bottomright']['y']

        confidence = result['confidence']
        label = result['label'] + " " + str(round(confidence, 3))

        # only show boxes that are above .1 confidence and for the label, bird
        if confidence > 0.45 and result['label'] == 'bird' :
            # draw boxes on images
            fontScale = min(im_width,im_height)/(600)
            newImage = cv2.rectangle(newImage, (xmin, ymax), (xmax, ymin), (255, 0, 157), 3)
            newImage = cv2.putText(newImage, label, (xmin, ymax-5), cv2.FONT_HERSHEY_SIMPLEX, fontScale, (153, 255, 255), 5, cv2.LINE_AA)

            # Optional: if mounted to Drive, export detection results to aves_det_crops_1000.tsv
            # Note: if performing detection on larger image batches, can break up files into multiple parts aves_det_crops_2000_a.tsv
            if os.path.exists('/content/drive/My Drive/fall19_smithsonian_informatics/aves_det_crops_1000.tsv'):
              with open('/content/drive/My Drive/fall19_smithsonian_informatics/aves_det_crops_1000.tsv', 'a') as out_file:
                  tsv_writer = csv.writer(out_file, delimiter='\t')
                  tsv_writer.writerow([image_url, im_height, im_width, 
                            xmin, ymin, xmax, ymax])
            
        else:
          print("No birds detected in {}.".format(image_url))
    return newImage

## Load in sample images and 'flow' them through the object detector
---
You can either **A) Load individual images in by URL**, or for large image batches or **B) Load multiple images from a text file of image URLs**. Other methods for importing to Google Colab are listed [here](https://colab.research.google.com/notebooks/io.ipynb#scrollTo=XDg9OBaYqRMd). 

**A) Load individual images in by URL**
Load in images by URL and run the image detector for all images. Plotted results include the image with bounding box around detected objects (birds), class type, and confidence score. Inference times are printed above images. If you "mounted" your Google Drive during "Installs", the bounding box coordinates will also be written to 'sample_crops_yolo.tsv'.

In [ ]:
image_urls = ["https://content.eol.org/data/media/7e/9c/7a/542.15445377044.jpg",
              "https://content.eol.org/data/media/81/1c/0d/542.7816025222.jpg",
              "https://content.eol.org/data/media/7e/3c/0b/542.10578857864.jpg"]

In [ ]:
for image_url in image_urls:
  image = url_to_image(image_url)

  # Use YOLO for object detection  
  # Record inference time
  start_time = time.time()
  result = tfnet.return_predict(image)
  end_time = time.time()

  # Plot and show detection boxes on images
  _, ax = plt.subplots(figsize=(10, 10))
  ax.imshow(boxing(image, result))

  # Display inference time above images
  plt.title('Inference time: {}'.format(format(end_time-start_time, '.2f')))

**B) Load multiple images (from EOL image URL bundles) through object detector**   
Load in multiple images from a text file of URLS and run the image detector for all images. Plotted results include the image with bounding box around detected objects (birds), class type, and confidence score. Inference times are printed above images. If you "mounted" your Google Drive during "Installs", the bounding box coordinates will also be written to 'sample_crops_yolo.tsv'.

In [ ]:
# For 1000 or 20000 image datasets, change link below
# 1000 Aves images
urls = 'https://editors.eol.org/other_files/bundle_images/files/images_for_Aves_breakdown_download_000001.txt'
# 20000 Aves images
#urls = 'https://editors.eol.org/other_files/bundle_images/files/images_for_Aves_20K_breakdown_download_000001.txt'
df = pd.read_csv(urls)
df.columns = ["link"]
pd.DataFrame.head(df)

In [ ]:
# Write header row of output crops file
# For 1000 or 20000 image datasets, change filename here and in "Prepare object detection functions and settings -> def boxing -> Export detection results" above
# Note: if performing detection on larger image batches, can break up files into multiple parts, ex: aves_det_crops_20000_a.tsv for df.iloc[0:5000].iterrows() below
with open('/content/drive/My Drive/fall19_smithsonian_informatics/aves_det_crops_1000.tsv', 'a') as out_file:
                  tsv_writer = csv.writer(out_file, delimiter='\t')
                  tsv_writer.writerow(["image_url", "im_height", "im_width", 
                            "xmin", "ymin", "xmax", "ymax"])

In [ ]:
# Loops through first 5 image urls from the text file
for i, row in df.head(5).itertuples(index=True, name='Pandas'):

# For ranges of rows or all rows, use the commands below
# Note: can be useful if running large image batches through in multiple parts
#for i, row in df.iloc[0:5000].iterrows():
#for i, row in df.iterrows():

  try:
    # Record inference time
    start_time = time.time()
    image_url = df.get_value(i, "link")
    image = url_to_image(image_url)
    # Detection
    result = tfnet.return_predict(image)
    end_time = time.time()
    # Draw boxes on images
    boxing(image, result)
    # Display progress message after each image
    print('Detection complete in {} of 1,000 images'.format(i+1))
  
  except:
    print('Error: check if web address {} is valid'.format(image_url))
  
  # Plot and show detection boxes on images
  # If running detection on >50 images, comment out this portion
  _, ax = plt.subplots(figsize=(10, 10))
  ax.imshow(boxing(image, result))
  plt.title('{}) Inference time: {}'.format(i+1, format(end_time-start_time, '.2f')))
  plt.close()

### Get inference info for test images to compare object detection model times for YOLO, SSD, and Faster-RCNN

In [ ]:
from PIL import Image
import os

# For exporting inference times
inf_time = []
img_urls = []
im_dims = []

# Loops through first 5 image urls from the text file
#for i, row in df.head(5).itertuples(index=True, name='Pandas'):
for i, row in df.head(145).itertuples(index=True, name='Pandas'):

  try:
    image_url = df.get_value(i, "link")
    image = url_to_image(image_url)
    # Record inference time
    start_time = time.time()
    # Detection
    result = tfnet.return_predict(image)
    end_time = time.time()
    # Draw boxes on images
    boxing(image, result)
    # Display progress message after each image
    print('Detection complete in {} of 145 images'.format(i+1))

    # Record inference time, image name and image dimensions to export
    inf_time.append(end_time-start_time)
    img_urls.append(image_url)
    im_dims.append(image.shape)
  
  except:
    print('Error: check if web address {} is valid'.format(image_url))
    
inf_times = pd.DataFrame(([inf_time, img_urls, im_dims]))
inf_times = inf_times.transpose()
inf_times.to_csv("aves_inference_times_yolo.csv", index=False, header=("time (sec)", "filepath", "image_dims (h, w, d)"))
print(inf_times.head())